# Energy Derivatives Pricing Demo

Price renewable energy-backed derivatives using real solar data.

In [ ]:
import subprocess, sys
print('📦 Installing package...')
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'git+https://github.com/Spectating101/spk-derivatives.git'])
print('✅ Ready!')

In [ ]:
from spk_derivatives import load_solar_parameters, BinomialTree, MonteCarloSimulator, calculate_greeks
import pandas as pd, numpy as np

print('Loading Taiwan solar data...')
params = load_solar_parameters(lat=24.99, lon=121.30, volatility_cap=2.0, volatility_method='log', cache=True)
print(f'Spot: ${params["S0"]:.4f}/kWh, Vol: {params["sigma"]:.2%}')

In [ ]:
core = {k: params[k] for k in ('S0', 'K', 'T', 'r', 'sigma')}

print('\n🌳 BINOMIAL PRICING')
bt = BinomialTree(**core, N=400, payoff_type='call')
price_bin = bt.price()
print(f'Call option: ${price_bin:.6f}/kWh')

In [ ]:
print('\n🎲 MONTE CARLO VALIDATION')
mc = MonteCarloSimulator(**core, num_simulations=20000, seed=42, payoff_type='call')
price_mc, low, high = mc.confidence_interval(0.95)
print(f'Call option: ${price_mc:.6f}/kWh')
print(f'95% CI: ${low:.6f} - ${high:.6f}')
print(f'Error: {abs(price_bin-price_mc)/price_mc*100:.2f}%')

In [ ]:
print('\n📊 THE GREEKS')
greeks_df = calculate_greeks(**core, pricing_method='binomial', N=100)
greeks = dict(zip(greeks_df['Greek'], greeks_df['Value']))
for g in ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho']:
    print(f'{g}: {greeks[g]:.4f}')

## Summary

✅ Pricing: Binomial & MC converge  
✅ Risk: Greeks quantify all sensitivities  
✅ Real data: NASA solar irradiance  

Energy-backed derivatives are real, measurable, and tradeable.